<a href="https://colab.research.google.com/github/stuart-lane/MachineLearning/blob/main/LogitSimulations2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import random

np.random.seed(123456)

DATA GENERATING PROCESS

In [2]:
n = 1000
x = np.random.randn(n)
u = np.random.logistic(0, 1, n)
beta0 = 0
beta1 = 1

ystar = beta0 + beta1 * x + u
y = (ystar > 0).astype(int)

#p = 1 / (1 + np.exp(-(beta0 + x*beta1 + u)))
#y = np.random.binomial(n = 1, p = p, size = n)

In [3]:
from sklearn.model_selection import train_test_split

u = u.reshape(-1, 1)
x = np.hstack((np.ones((n, 1)), x.reshape(-1, 1)))
y = y.reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 123)

train_df_keys = {'constant': x_train[:,0].flatten(), 'X1': x_train[:,1].flatten(), 'y': y_train.flatten()}
train_df = pd.DataFrame(train_df_keys)

LOGISTIC REGRESSION WITH SKLEARN

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
model = LogisticRegression(solver = 'lbfgs', max_iter = 1000)  # You can change the solver and max_iter as needed
model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(max_iter=1000)

In [6]:
intercept = model.intercept_[0]
coefficient = model.coef_[0][1]

In [7]:
print(f"Intercept: {intercept}")
print(f"Coefficient for 'Regressor': {coefficient}")

Intercept: -0.12587614423279017
Coefficient for 'Regressor': 0.9506309771391525


STANDARD LOGISTIC REGRESSION

In [12]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, learning_rate, num_iterations):
    m, n = X.shape
    theta = np.zeros((n, 1))

    for i in range(num_iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / m
        theta -= learning_rate * gradient
        if (i+1) % 100 == 0:
          print(f"Iteration: {i+1} || Gradients: {gradient[0][0]} and {gradient[1][0]}")
          print(f"Estimated Parameters (Weights): {theta[0][0]} and {theta[1][0]}")

    return theta

learning_rate = 0.1
num_iterations = 1000

theta = logistic_regression(x_train, y_train, learning_rate, num_iterations)

Iteration: 100 || Gradients: 0.0024484652999759928 and -0.024826645218115048
Estimated Parameters (Weights): -0.11329131281618507 and 0.81057807398994
Iteration: 200 || Gradients: 0.0003652894496281411 and -0.00469650720890295
Estimated Parameters (Weights): -0.12381547601229506 and 0.9287924239136948
Iteration: 300 || Gradients: 6.741809573183612e-05 and -0.0009668692203220522
Estimated Parameters (Weights): -0.1255479423812796 and 0.9521417298211473
Iteration: 400 || Gradients: 1.3323826691407498e-05 and -0.00020236601955596283
Estimated Parameters (Weights): -0.1258777871830009 and 0.9569891455894065
Iteration: 500 || Gradients: 2.7082526901361924e-06 and -4.249818941969404e-05
Estimated Parameters (Weights): -0.12594378392045522 and 0.9580054499044263
Iteration: 600 || Gradients: 5.583272185494259e-07 and -8.93091069069759e-06
Estimated Parameters (Weights): -0.1259572801211017 and 0.9582189541023278
Iteration: 700 || Gradients: 1.1601941852526498e-07 and -1.8770430608018932e-06
Es

AUTOGRAD

In [13]:
import torch
from torch.optim import Adam
from torch.optim import LBFGS

In [14]:
train_df_keys = {'constant': x_train[:,0].flatten(), 'X1': x_train[:,1].flatten(), 'y': y_train.flatten()}

train_df = pd.DataFrame(train_df_keys)

In [18]:
x = torch.cat((torch.tensor(train_df[["constant", "X1"]].values, dtype=torch.float64),), dim=1)
x.requires_grad = True

y = torch.tensor(train_df["y"].values, dtype = torch.float64)
y.requires_grad = True
y = y.view(-1, 1)

beta_initial = np.array([0, 0])
beta = torch.zeros(2, 1, dtype = torch.float64)
beta.requires_grad = True

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    dev = "cuda"
else:
    dev = "cpu"

print(f"Running on: {dev}")

# Define the loss function
def ll(beta):
    xbetahat = torch.mm(x, beta)
    yhat = torch.sigmoid(xbetahat)
    loss = - torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    return loss

# Set up the Adam optimizer
learning_rate = 0.1
optimizer = LBFGS([beta], lr = learning_rate)

# Define a function to calculate the loss and update model parameters
def calculate_loss():
    optimizer.zero_grad()
    value = ll(beta)
    value.backward()
    return value

num_iter = 100
for i in range(1, num_iter + 1):
  logl = optimizer.step(calculate_loss)
  logl_value = logl.detach().item()
  estimated_weights = beta.detach().numpy()
  if (i + 1) % 10 == 0:
    print(f"Iteration {i + 1} || Log-value {logl_value}")
    print(f"Estimated weights: {estimated_weights}")

estimated_weights_LBFGS = beta.detach().numpy()
print("Estimated Parameters (Weights):", estimated_weights_LBFGS)

Running on: cpu
Iteration 10 || Log-value 478.8465763149336
Estimated weights: [[-0.12596   ]
 [ 0.95826996]]
Iteration 20 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 30 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 40 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 50 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 60 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 70 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 80 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 90 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Iteration 100 || Log-value 478.8465763127937
Estimated weights: [[-0.12596042]
 [ 0.95827299]]
Estimated Parameters (Weights): [[-0.125960

NEURAL NETWORK LOGIT

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [21]:
class LogitModel(nn.Module):
    def __init__(self, input_dim):
        super(LogitModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Single output unit

    def forward(self, x):
        x = x.to(self.linear.weight.dtype)  # Cast input to the same data type as the model's parameters
        x = self.linear(x)
        x = torch.sigmoid(x)  # Apply sigmoid activation for binary classification
        return x

x = torch.tensor(x_train, dtype = torch.float64)
x.requires_grad = True

y = torch.tensor(y_train, dtype = torch.float64)
y.requires_grad = True
y = y.reshape(-1, 1)

beta = torch.randn(x.shape[1], 1, dtype = torch.float64)
beta.requires_grad = True

input_dim = x.shape[1]

model = LogitModel(input_dim)
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.1)

epochs = 5000

# Training loop
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Zero the gradients

    outputs = model(x)
    outputs = outputs.float()
    y = y.float()

    loss = loss_function(outputs, y)
    loss.backward()
    optimizer.step()
    if epoch % (epochs / 10) == 0:
        print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")

# View the learned parameters
print("Learned parameters:")
print(f"Model weights: {model.linear.weight[0][0]}, {model.linear.weight[0][1]}") # these are the parameters of interest
print(f"Model bias:{model.linear.bias[0]}")

Epoch [0/5000], Loss: 0.7002
Epoch [500/5000], Loss: 0.5986
Epoch [1000/5000], Loss: 0.5986
Epoch [1500/5000], Loss: 0.5986
Epoch [2000/5000], Loss: 0.5986
Epoch [2500/5000], Loss: 0.5986
Epoch [3000/5000], Loss: 0.5986
Epoch [3500/5000], Loss: 0.5986
Epoch [4000/5000], Loss: 0.5986
Epoch [4500/5000], Loss: 0.5986
Learned parameters:
Model weights: -0.15777356922626495, 0.9582759141921997
Model bias:0.031812842935323715


WORKING PYTORCH LOGISTIC REGRESSION

In [22]:
import torch

x = torch.tensor(x_train, dtype = torch.float64)
x.requires_grad = True

y = torch.tensor(y_train, dtype = torch.float64)
y.requires_grad = True
y = y.reshape(-1, 1)

beta = torch.randn(x.shape[1], 1, dtype = torch.float64)
beta.requires_grad = True

# Initialize parameters
learning_rate = 0.01
num_iterations = 10

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

# Training loop
for i in range(num_iterations):
    xbetahat = torch.matmul(x, beta)
    yhat = sigmoid(xbetahat)

    loss = -torch.sum(y * torch.log(yhat) + (1 - y) * torch.log(1 - yhat))
    loss.backward()

    with torch.no_grad():
        beta -= learning_rate * beta.grad
    beta.grad.zero_()

    print(f'Iteration {i+1}, Loss: {loss.item()}')

# Get the learned coefficients
learned_beta = beta.detach().numpy()

print(f'Learned Logistic Regression Coefficients (Beta): {learned_beta}')

Iteration 1, Loss: 839.2450018031193
Iteration 2, Loss: 599.1861353337963
Iteration 3, Loss: 492.3846833469495
Iteration 4, Loss: 479.2939064387473
Iteration 5, Loss: 479.0372641955145
Iteration 6, Loss: 478.92708631838406
Iteration 7, Loss: 478.8807817587528
Iteration 8, Loss: 478.86109068399554
Iteration 9, Loss: 478.85274167867703
Iteration 10, Loss: 478.8491941862245
Learned Logistic Regression Coefficients (Beta): [[-0.12229851]
 [ 0.9585038 ]]
